# Chess-Bot

## Setup

In [1]:
%load_ext autoreload
%autoreload 2

import chessbot as cb

## Environment

In [2]:
cb.environment.create()

Dataset archive file found at /home/wumphlett/personal/Chess-Bot/data/CCRL-4040.[1505357].pgn.7z
Dataset file found at /home/wumphlett/personal/Chess-Bot/data/CCRL-4040.[1505357].pgn
Dataset pkl files found at /home/wumphlett/personal/Chess-Bot/data
Dataset val pkl files found at /home/wumphlett/personal/Chess-Bot/data
Done


## Pos2Vec

In [3]:
x_train, x_val = cb.dataset.pos2vec_dataset()

In [ ]:
pos2vec = cb.model.train_pos2vec(x_train, x_val)

Training layer 1/4
Epoch 1/200
40000/40000 [==============================] - 64s 2ms/step - loss: 0.0178 - lr: 0.0050
Epoch 2/200
40000/40000 [==============================] - 67s 2ms/step - loss: 0.0262 - lr: 0.0049
Epoch 3/200
40000/40000 [==============================] - 65s 2ms/step - loss: 0.0303 - lr: 0.0048
Epoch 4/200
40000/40000 [==============================] - 64s 2ms/step - loss: 0.0323 - lr: 0.0047
Epoch 5/200
40000/40000 [==============================] - 64s 2ms/step - loss: 0.0333 - lr: 0.0046
Epoch 6/200
40000/40000 [==============================] - 67s 2ms/step - loss: 0.0334 - lr: 0.0045
Epoch 7/200
40000/40000 [==============================] - 65s 2ms/step - loss: 0.0328 - lr: 0.0044
Epoch 8/200
40000/40000 [==============================] - 64s 2ms/step - loss: 0.0327 - lr: 0.0043
Epoch 9/200
40000/40000 [==============================] - 63s 2ms/step - loss: 0.0313 - lr: 0.0043
Epoch 10/200
40000/40000 [==============================] - 64s 2ms/step - loss: 

40000/40000 [==============================] - 64s 2ms/step - loss: 5.1002e-04 - lr: 9.9324e-04
Epoch 82/200
40000/40000 [==============================] - 64s 2ms/step - loss: 4.6393e-04 - lr: 9.7338e-04
Epoch 83/200
40000/40000 [==============================] - 65s 2ms/step - loss: 4.5789e-04 - lr: 9.5391e-04
Epoch 84/200
40000/40000 [==============================] - 64s 2ms/step - loss: 3.9870e-04 - lr: 9.3483e-04
Epoch 85/200
40000/40000 [==============================] - 64s 2ms/step - loss: 4.0955e-04 - lr: 9.1614e-04
Epoch 86/200
40000/40000 [==============================] - 64s 2ms/step - loss: 4.1385e-04 - lr: 8.9781e-04
Epoch 87/200
40000/40000 [==============================] - 65s 2ms/step - loss: 3.4045e-04 - lr: 8.7986e-04
Epoch 88/200
40000/40000 [==============================] - 63s 2ms/step - loss: 3.3753e-04 - lr: 8.6226e-04
Epoch 89/200
40000/40000 [==============================] - 63s 2ms/step - loss: 3.6225e-04 - lr: 8.4502e-04
Epoch 90/200
40000/40000 [======

40000/40000 [==============================] - 62s 2ms/step - loss: 5.3925e-25 - lr: 2.2273e-04
Epoch 156/200
40000/40000 [==============================] - 63s 2ms/step - loss: 5.3925e-25 - lr: 2.1828e-04
Epoch 157/200
40000/40000 [==============================] - 64s 2ms/step - loss: 5.3925e-25 - lr: 2.1391e-04
Epoch 158/200
40000/40000 [==============================] - 63s 2ms/step - loss: 5.3925e-25 - lr: 2.0964e-04
Epoch 159/200
40000/40000 [==============================] - 63s 2ms/step - loss: 5.3925e-25 - lr: 2.0544e-04
Epoch 160/200
40000/40000 [==============================] - 63s 2ms/step - loss: 5.3925e-25 - lr: 2.0133e-04
Epoch 161/200
40000/40000 [==============================] - 64s 2ms/step - loss: 5.3925e-25 - lr: 1.9731e-04
Epoch 162/200
40000/40000 [==============================] - 63s 2ms/step - loss: 5.3925e-25 - lr: 1.9336e-04
Epoch 163/200
40000/40000 [==============================] - 65s 2ms/step - loss: 5.3925e-25 - lr: 1.8949e-04
Epoch 164/200
40000/4000

In [ ]:
import numpy as np

ex = np.reshape(x_train[0][0][0], (1, 773))

print("ex")
print(ex)

print("enc")
print(pos2vec(ex)[0])

## Sandbox